<a href="https://colab.research.google.com/github/mhanson16/Home_Sales/blob/main/Home_Sales_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.3.1'
spark_version = 'spark-3.4.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:5 http://archive.ubuntu.com/ubuntu focal-updates InRelease
Hit:6 http://security.ubuntu.com/ubuntu focal-security InRelease
Get:7 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:8 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:9 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:11 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Fetched 108 kB in 2s (45.3 kB/s)
Reading package lists... Done


In [5]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [8]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"


spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True)


In [9]:
# 2. Create a temporary view of the DataFrame.
df.createOrReplaceTempView('home_sales')
spark.sql("SELECT * FROM home_sales LIMIT 10").show()



+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [10]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
# To answer this question year must be its own column
from pyspark.sql.functions import year
df.select(year(df["date"]))

#Adding year column to date frame
df = df.withColumn("year", year(df['date']))

# Create a temporary view of the DataFrame.
df.createOrReplaceTempView('home_sales')

#SQL Query
spark.sql("""Select year, round(avg(price),2) 
From home_sales 
Where bedrooms == 4 
Group By year""").show()



+----+--------------------+
|year|round(avg(price), 2)|
+----+--------------------+
|2022|           296363.88|
|2019|            300263.7|
|2020|           298353.78|
|2021|           301819.44|
+----+--------------------+



In [11]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
spark.sql("""Select date_built, round(avg(price),2) 
From home_sales 
Where bedrooms == 3 And bathrooms == 3 
Group By date_built""").show()



+----------+--------------------+
|date_built|round(avg(price), 2)|
+----------+--------------------+
|      2016|           290555.07|
|      2012|           293683.19|
|      2017|           292676.79|
|      2014|           290852.27|
|      2013|           295962.27|
|      2011|           291117.47|
|      2015|            288770.3|
|      2010|           292859.62|
+----------+--------------------+



In [12]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
spark.sql("""Select date_built, round(avg(price),2) 
From home_sales 
Where bedrooms == 3 And bathrooms == 3 And floors == 2 And sqft_lot >= 2000
Group By date_built""").show()



+----------+--------------------+
|date_built|round(avg(price), 2)|
+----------+--------------------+
|      2016|            287812.6|
|      2012|           295858.68|
|      2017|           282026.59|
|      2014|           294195.13|
|      2013|           295142.13|
|      2011|           281413.45|
|      2015|           291788.36|
|      2010|           280447.23|
+----------+--------------------+



In [13]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()

spark.sql("""Select view, round(avg(price),2) 
From home_sales 
Where price >= 350000
Group By view""").show()



print("--- %s seconds ---" % (time.time() - start_time))

+----+--------------------+
|view|round(avg(price), 2)|
+----+--------------------+
|   7|           403005.77|
|  51|           788128.21|
|  15|            404673.3|
|  54|           798684.82|
|  11|           399548.12|
|  29|           397771.65|
|  69|           750537.94|
|  42|            396964.5|
|  87|           1072285.2|
|  73|           752861.18|
|  64|           767036.67|
|   3|            398867.6|
|  30|            397862.0|
|  34|           401419.75|
|  59|            791453.0|
|   8|           398592.71|
|  28|           402124.62|
|  22|           402022.68|
|  85|          1056336.74|
|  16|           399586.53|
+----+--------------------+
only showing top 20 rows

--- 1.0887353420257568 seconds ---


In [14]:
# 7. Cache the the temporary table home_sales.
spark.sql("cache table home_sales")

DataFrame[]

In [15]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [16]:
# 9. Using the cached data, run the query that filters out the view ratings with average price 
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()

spark.sql("""Select view, round(avg(price),2) 
From home_sales 
Where price >= 350000
Group By view""").show()


print("--- %s seconds ---" % (time.time() - start_time))


+----+--------------------+
|view|round(avg(price), 2)|
+----+--------------------+
|   7|           403005.77|
|  51|           788128.21|
|  15|            404673.3|
|  54|           798684.82|
|  11|           399548.12|
|  29|           397771.65|
|  69|           750537.94|
|  42|            396964.5|
|  87|           1072285.2|
|  73|           752861.18|
|  64|           767036.67|
|   3|            398867.6|
|  30|            397862.0|
|  34|           401419.75|
|  59|            791453.0|
|   8|           398592.71|
|  28|           402124.62|
|  22|           402022.68|
|  85|          1056336.74|
|  16|           399586.53|
+----+--------------------+
only showing top 20 rows

--- 0.5517776012420654 seconds ---


The run time after using the cached data was about .536958 seconds faster.

In [17]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data 
df.write.partitionBy("date_built").mode("overwrite").parquet("delayed_partitioned")


In [18]:
# 11. Read the parquet formatted data.
p_df_p=spark.read.parquet('delayed_partitioned')

In [19]:
# 12. Create a temporary table for the parquet data.
p_df_p.createOrReplaceTempView('home_sales')
spark.sql("SELECT * FROM home_sales LIMIT 10").show()

+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----+----------+
|                  id|      date| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|year|date_built|
+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----+----------+
|2ed8d509-7372-46d...|2021-08-06|258710|       3|        3|       1918|    9666|     1|         0|  25|2021|      2015|
|941bad30-eb49-4a7...|2020-05-09|229896|       3|        3|       2197|    8641|     1|         0|   3|2020|      2015|
|c797ca12-52cd-4b1...|2019-06-08|288650|       2|        3|       2100|   10419|     2|         0|   7|2019|      2015|
|0cfe57f3-28c2-472...|2019-10-04|308313|       3|        3|       1960|    9453|     2|         0|   2|2019|      2015|
|d715f295-2fbf-4e9...|2021-05-17|391574|       3|        2|       1635|    8040|     2|         0|  10|2021|      2015|
|a18515a2-86f3-46b...|2022-02-18|419543|

In [20]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000 
# with the parquet DataFrame. Round your average to two decimal places. 
# Determine the runtime and compare it to the cached version.

start_time = time.time()

spark.sql("""Select view, round(avg(price),2) 
From home_sales 
Where price >= 350000
Group By view""").show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+--------------------+
|view|round(avg(price), 2)|
+----+--------------------+
|   7|           403005.77|
|  51|           788128.21|
|  15|            404673.3|
|  54|           798684.82|
|  11|           399548.12|
|  29|           397771.65|
|  69|           750537.94|
|  42|            396964.5|
|  73|           752861.18|
|  87|           1072285.2|
|  64|           767036.67|
|   3|            398867.6|
|  30|            397862.0|
|  34|           401419.75|
|  59|            791453.0|
|   8|           398592.71|
|  28|           402124.62|
|  22|           402022.68|
|  85|          1056336.74|
|  35|           401934.21|
+----+--------------------+
only showing top 20 rows

--- 0.9350125789642334 seconds ---


The parquet DataFrame was slower .383235 seconds than the cached.

In [21]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table home_sales")

DataFrame[]

In [22]:
# 15. Check if the home_sales is no longer cached

spark.catalog.isCached("home_sales")


False